In [ ]:
name = "wwwwwwwwwwwwwwwwwwwwwwwwww"

gdrive_path = f"/content/drive/Othercomputers/내 노트북/gdrive/dppos/{name}"


try:
    from google.colab import drive
    drive.mount('/content/drive')
    !cp    "{gdrive_path}/world_samples.py" world_samples.py
    !cp -r "{gdrive_path}/drivingppo"       drivingppo/
    !mkdir -p "{gdrive_path}/checks" "{gdrive_path}/logs"
    !pip uninstall -y gym
    !pip install gymnasium stable-baselines3
except ModuleNotFoundError:
    print('코랩아님')


def colabflush():
    try:
        from google.colab import drive
        !mv checks/* "{gdrive_path}/checks/"
        !mv logs/*   "{gdrive_path}/logs/"
    except ModuleNotFoundError:
        pass


텐서보드 log 확인

```
tensorboard --logdir ./logs/
```

[http://localhost:6006](http://localhost:6006)

In [ ]:
import time
from drivingppo.common import set_seed
from drivingppo.environment import WorldEnv, get_path_features
from drivingppo.model import create_model, train, linear_schedule, evaluate
from drivingppo.model import (
    NoFeaturesExtractor,
    VMLPFeaturesExtractor,
    Shwp1FeaturesExtractor,
)
from world_samples import gen_1t, gen_2t, gen_3t, gen_3l

# from IPython.display import clear_output

#########################################################

t0 = time.strftime('%y%m%d-%H%M')
tb_log = True

policy_kwargs=dict(
    features_extractor_class=NoFeaturesExtractor,
    features_extractor_kwargs=dict(
        # conv_out_channels=16,
    ),
    net_arch=dict(
        pi=[512, 512],
        vf=[512, 512, 256]
    )
)

#########################################################

print('PATH_FEATURE_FUNCTION: ', get_path_features)

def a(seed):
    run_name = f'{name}-{t0}-{seed}'
    print('RUN:', run_name)

    count_max = 20

    model = create_model(
        policy_kwargs=policy_kwargs,
    )

    print(f'=== 1단계')
    count = 0
    while True:
        count += 1
        set_seed(seed)
        model = train(
            model=model,
            # save_path=f'{prefix}-{seed}--1',
            gen_env=lambda: WorldEnv(world_generator=gen_1t, time_gain_limit=9_000, time_gain_per_waypoint_rate=0, collision_ending=False),
            tb_log=tb_log,
            run_name=run_name,
            steps=4096,
            lr=9e-4,
            gamma=0.8,
            progress_display=None,
            seed=seed
        )
        metrics = evaluate(model, gen_1t, csv_path='', episode_num=100, verbose=False, print_result=False)
        endings = metrics['ending/type']
        success_rate = (endings.count('arrived') + endings.count('timeout')) / len(endings)
        print(f'=== 1단계 ({count}) 성공률: {success_rate*100:.0f}%')
        if success_rate >= 0.5 - 0.0001:
            break
        elif count > count_max:
            break
    if count > count_max:
        print(f'=== 1단계 통과 못함. 그만.')
        return

    print(f'=== 2단계')
    count = 0
    while True:
        count += 1
        set_seed(seed)
        model = train(
            model=model,
            # save_path=f'{prefix}-{seed}--2',
            gen_env=lambda: WorldEnv(world_generator=gen_2t, time_gain_limit=12_000, time_gain_per_waypoint_rate=0, collision_ending=False),
            tb_log=tb_log,
            run_name=run_name,
            steps=4096,
            lr=5e-4,
            gamma=0.95,
            progress_display=None,
            seed=seed
        )
        metrics = evaluate(model, gen_2t, csv_path='', episode_num=100, verbose=False, print_result=False)
        endings = metrics['ending/type']
        success_rate = (endings.count('arrived') + endings.count('timeout')) / len(endings)
        print(f'=== 2단계 ({count}) 성공률: {success_rate*100:.0f}%')
        if success_rate >= 0.95 - 0.0001:
            break
        elif count > count_max:
            break
    if count > count_max:
        print(f'=== 2단계 통과 못함. 그만.')
        return

    print('=== 3단계')
    set_seed(seed)
    model = train(
        model=model,
        gen_env=lambda: WorldEnv(world_generator=gen_3t, time_gain_limit=20_000, time_gain_per_waypoint_rate=0),
        tb_log=tb_log,
        run_name=run_name,
        # save_freq=409_6000//10//4,
        steps=409_6000,
        lr=linear_schedule(3e-4),
        gamma=1,
        seed=seed
    )
    model = train(  # 학습률 스케줄러 안 쓰도록 모델 저장
        model=model,
        save_path=f'{name}-{seed}',
        gen_env=lambda: WorldEnv(world_generator=gen_3t),
        steps=0,
        lr=0.0,
    )
    set_seed(seed)
    evaluate(model, gen_3l, episode_num=100, csv_path='', verbose=False, print_result=True)

    colabflush()

    return model

colabflush()
model = a(100)
model = a(200)
model = a(300)
model = a(400)
model = a(500)

In [ ]:
# # 학습률 스케줄 쓰면 코랩에서 돌린 게 내 컴에서 실행이 안 됨. 그거만 덮어쓰고 다시 저장.
# from stable_baselines3 import PPO

# path = 'checks/wwwwwwwwwwwwwww'

# model = PPO.load(
#     path=path,
#     learning_rate=0.0,
# )
# model.save(path)

| `log_std` | 실제 표준편차 σ | 탐색 수준 | 비고 |
| --- | --- | --- | --- |
| 0 | 1.0 | 매우 높음 | 행동 공간 전체를 휘젓고 다니는 수준. 초기 학습 시 강한 탐색이 필요할 때 사용. |
| -0.5 | ≈ 0.6 | 높음 | 행동의 변동폭이 큼. 기존 정책이 굳었을 때 강제 리셋용으로 권장되는 값. |
| -1 | ≈ 0.37 | 보통 | 안정적인 주행과 탐색이 공존하는 수준. |
| -2 | ≈ 0.13 | 낮음 | 어느 정도 최적화가 진행된 상태. 정밀한 제어를 시작함. |
| \~-3.0 | \< 0.05 | 거의 없음 | 수렴 완료 단계. 거의 결정론적(Deterministic)인 행동을 보임. |